In [1]:
import sys, os, re, csv, numpy as np, pandas as pd
import string
from collections import Counter
from nltk import word_tokenize
from tqdm import tqdm

In [4]:
print('loading embeddings vectors')
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(' ')) for o in open('/home/tcs/pavancs/glove/glove.840B.300d.txt',encoding="utf8"))

loading embeddings vectors


In [5]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')

In [6]:
test.columns

Index(['id', 'comment_text'], dtype='object')

In [19]:
embeddings_index ={}

In [7]:
test.head()

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you wi...
1,0000247867823ef7,from rfc the title is fine as it is imo
2,00013b17ad220c46,sources zawe ashton on lapland
3,00017563c3f7919a,if you have a look back at the source the info...
4,00017695ad8997eb,i do not anonymously edit articles at all


In [9]:
##convert that into lower case.
X_train = train["comment_text"].str.lower()
X_test = test["comment_text"].str.lower()

In [10]:
allwords = []
for i in embeddings_index:
    allwords.append(i)

In [11]:
embeddings_index['cheatsheet']

array([-0.046267 , -0.52833  ,  0.90497  ,  0.65461  , -0.39479  ,
        0.8632   , -0.13329  ,  0.21827  ,  0.15443  , -1.632    ,
       -0.14961  , -0.30423  ,  0.22353  ,  0.54915  ,  0.15814  ,
        0.0079292, -0.26355  , -0.27593  ,  0.44409  , -0.17517  ,
       -0.276    ,  0.26104  , -0.32568  , -0.064238 , -0.79084  ,
        0.65574  ,  0.12605  , -0.28961  ,  0.094027 , -0.381    ,
        0.30555  , -0.20541  , -0.33254  ,  0.11742  ,  0.093219 ,
        0.18444  , -0.45595  , -0.45334  , -0.024553 , -0.42731  ,
       -0.73834  , -0.0081092,  0.13698  , -0.60227  , -0.83236  ,
        0.38058  ,  0.019084 ,  0.59652  ,  0.027366 , -0.86875  ,
       -0.30201  , -0.26196  ,  0.0046073, -0.53407  ,  0.60293  ,
       -0.34597  ,  0.80496  , -0.1221   ,  0.096038 , -0.31023  ,
       -0.40489  , -0.25225  , -0.13741  , -0.054878 ,  0.10366  ,
        0.90815  , -0.29922  ,  0.40268  ,  0.38758  ,  0.3715   ,
       -0.28129  , -0.70797  ,  0.93636  ,  0.14087  , -0.0685

In [12]:
len(allwords)

2196016

In [13]:
#first 5 words in the embedding matrix
allwords[:5]

[',', '.', 'the', 'and', 'to']

In [14]:
#spell checking convert the word which is not available in the vector file into one of the word which in vector file by changing
#the word into nearest two editable word.
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())
#WORDS = Counter(words(open('big.txt').read()))
WORDS = Counter(allwords)
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [15]:
correction('vandalismsss')

'vandalisms'

In [16]:
uniq = []

In [17]:
for i in X_test:
    print(i)
    break

yo bitch ja rule is more succesful then you will ever be whats up with you and hating you sad mofuckas i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother too fuckin white boys get things right next time


In [18]:
correct_train = []
uniq = []
count = 0
for i in X_test:
    x = word_tokenize(str(i))
    tok = []
    for j in x:
        try:
            embeddings_index[j]
        except:
                uniq.append(j)

In [157]:
test_sub = test[0:1000]

In [ ]:
correct_train = []
count = 0
for i in X_train:
    x = word_tokenize(i)
    tok = []
    for j in x:
        try:
            embeddings_index[j]
        except:
            j = correction(j)
        j = str(j)
        tok.append(j)
    tok = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tok]).strip()
    correct_train.append(tok)
    count += 1
    if(count%100 == 0):
        print(count)
        

100
200
300


In [21]:
len(uniq)

306026

In [22]:
newlist = Counter(x for x in uniq)

In [23]:
finallist = newlist.most_common(15000)

In [24]:
finallist[0:10]

[('youfuck', 3276),
 ('و', 1811),
 ('npov', 1468),
 ('क', 1251),
 ('ㅂㄱㅇ', 1230),
 ('در', 1204),
 ('که', 1054),
 ('र', 1039),
 ('dixz', 1000),
 ('র', 991)]

In [25]:
finallist1=[]
for i in finallist:
    if len(i[0])<10:
        finallist1.append(i)

In [27]:
len(finallist1)

12913

In [28]:
corrected_words = []
k=0
for i in finallist1:
    k=k+1
    corrected_words.append(correction(i[0]))
    if k%100==0:
        print(k)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900


In [29]:
len(corrected_words)
corrected_words[:5]
finallist1[:5]

[('youfuck', 3276), ('و', 1811), ('npov', 1468), ('क', 1251), ('ㅂㄱㅇ', 1230)]

In [30]:
corrected_words[:5]

['dogfuck', 'r', 'pov', 'r', 'ㅂㄱㅇ']

In [31]:
finallist1[634:637]

[('km²', 31), ('یافته', 31), ('名', 31)]

In [32]:
X_test.shape

(153164,)

In [33]:
sub=X_test
# sub=pd.DataFrame(sub)
# sub=sub['comment_text']
# sub

In [34]:
%%time
for j,k in zip(finallist1,corrected_words):
    sub=pd.Series(sub).str.replace(j[0], k)

CPU times: user 26min 30s, sys: 1.96 s, total: 26min 32s
Wall time: 26min 35s


In [164]:
len(finallist1)

12913

In [35]:
sub1=pd.DataFrame(sub)
sub1[:5]

,comment_text
0,yo bitch ja rule is more succesful then you wi...
1,from rfc the title is fine as it is imo
2,sources zawe ashton on lapland
3,if you have a look back at the source the info...
4,i do not anonymously edit articles at all


In [36]:
sub1['comment_text'].isnull().sum()

66

In [37]:
test.isnull().sum()

id               0
comment_text    66
dtype: int64

In [38]:
sub1=sub1.fillna('no comment')

In [39]:
sub1['comment_text'][sub1['comment_text'].str.contains('mothjer')]

Series([], Name: comment_text, dtype: object)

In [41]:
test['comment_text']=sub1['comment_text']

In [42]:
test.head()

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you wi...
1,0000247867823ef7,from rfc the title is fine as it is imo
2,00013b17ad220c46,sources zawe ashton on lapland
3,00017563c3f7919a,if you have a look back at the source the info...
4,00017695ad8997eb,i do not anonymously edit articles at all


In [43]:
test.to_csv('spell_test.csv',index=False)

In [ ]:
sub1['id']=train['id']

In [144]:
train1['comment_text'][train1['comment_text'].str.contains('mothjer')]
#train.iloc[43]

2249      you are a mothjer fucker cocksucker you are a ...
64084     you are a mothjer fucker cocksucker you are a ...
92050     you are a mothjer fucker cocksucker you are a ...
133478    you are a mothjer fucker cocksucker you are a ...
154181    tyou are a mothjer fucker cocksucker you are a...
Name: comment_text, dtype: object

In [25]:
newlist = Counter(x for x in uniq)

In [37]:
corrected_words

['npv',
 '~~~~',
 'styles',
 '|',
 'fffff',
 'g.',
 'widths',
 '3rm',
 'zstyle',
 '==',
 'fuckee',
 'yourselfto',
 'preceding',
 'border:1px',
 'mother',
 'fgt',
 '//en.wikipedia.org/w/index.php',
 "d'image",
 'philippineslong',
 'cellpadding',
 'offfice',
 'pro-assad.hanibal911you',
 'bitches.fuck',
 'aeneid',
 'rice',
 'three-event',
 'fuck',
 'v',
 'pagedelete',
 'classr',
 'notrhbysouthbanof',
 'mainpage',
 'a/i',
 'admin-c',
 'criminalwar',
 'bunksteve',
 '|',
 'marcolfuck',
 'boymamas',
 'penis/////small',
 'wthe',
 'edgar181',
 '//en.wikipedia.org/wiki/wikipedia',
 'tommy2010',
 'securityfuck',
 'edit-warring',
 'bastered==bastered',
 'youbollocks',
 'fart.china',
 '//en.wikipedia.org/wiki/user_talk',
 'concernthanks',
 'ancestryfuck-off-jewish',
 'www.youtube.com/watch',
 'cellspacing',
 'j.delanoy',
 'deleted.this',
 'withlove',
 'situs',
 'fan-1967',
 'centraliststupid',
 'user_table',
 'nhrhs2010',
 'wp:3rr',
 'created/took',
 'supertall',
 'neil',
 'bleachanhero',
 'andsaid

In [27]:
sorted(newlist.items(),reverse = True)

[('🙈🙉🙊', 1),
 ('😜', 16),
 ('😔', 1),
 ('😏', 1),
 ('😉..', 1),
 ('😉', 15),
 ('😅', 1),
 ('😄', 1),
 ('😂😂😂😂', 1),
 ('😂😂😂', 3),
 ('😀', 1),
 ('📞📧', 2),
 ('💬', 1),
 ('💩', 1),
 ('💜', 1),
 ('👍', 1),
 ('🎤', 1),
 ('🎄', 3),
 ('🍌', 7),
 ('🍁', 11),
 ('𒁳', 38),
 ('𐌴𐌹', 1),
 ('𐌲', 1),
 ('𐌰𐌿', 1),
 ('𐌰𐌹', 1),
 ('��0000', 2),
 ('�ivory', 1),
 ('�columbia', 1),
 ('�', 3),
 ('ﾉ', 1),
 ('･∀･', 1),
 ('｡◕‿◕｡', 18),
 ('ｗｗｗ．ａｎｏｎｔａｌｋ．ｃｏｍ', 1),
 ('－but', 1),
 ('，現今因主要國土位置或政治因素而被通稱為「臺灣」', 1),
 ('，為亞洲現有最早實行共和立憲制度的國家', 1),
 ('，成立於1912年1月1日，於1971年前被廣泛承認代表中國', 1),
 ('（warau', 1),
 ('（talk）', 6),
 ('\ufeffsensual', 1),
 ('\ufeff', 1),
 ('ﬂute', 1),
 ('\uf738\uf738s', 1),
 ('\uf737\uf735\uf730', 1),
 ('\uf736\uf734', 1),
 ('\uf733\uf736\uf730', 1),
 ('\uf732\uf735\uf730', 1),
 ('\uf732\uf730\uf730', 1),
 ('\uf731\uf739\uf734\uf731', 1),
 ('\uf731\uf737\uf737s', 1),
 ('\uf701', 1),
 ('\uf6fc', 1),
 ('\uf6de', 1),
 ('\uf6d9', 3),
 ('\uf0b7', 1),
 ('\uf0a7', 2),
 ('\uf04a', 1),
 ('\uf03dtabfounder', 1),
 ('\uf03dtabex', 1),

In [29]:
finallist = newlist.most_common(2000)

In [34]:
flist = []
for i in finallist:
    flist.append(i[0])

In [35]:
flist

['npov',
 '~~~~',
 'style=',
 '|-',
 'f5fffa',
 '..',
 'width=',
 '3rr',
 '|style=',
 '====',
 'fucksex',
 'yourselfgo',
 '—preceding',
 'border:1px',
 'mothjer',
 'fggt',
 '//en.wikipedia.org/w/index.php',
 "'image",
 'philippineslong',
 'cellpadding=',
 'offfuck',
 'pro-assad.hanibal911you',
 'bitches.fuck',
 'deneid',
 'rice=',
 'three-revert',
 "'fuck",
 '\u200e',
 'pagedelete',
 '|class=',
 'notrhbysouthbanof',
 'mainpagebg',
 'an/i',
 'admin-',
 'criminalwar',
 'bunksteve',
 '||',
 'marcolfuck',
 'boymamas',
 'penis/////small',
 "'the",
 'edgar181',
 '//en.wikipedia.org/wiki/wikipedia',
 'tommy2010',
 'securityfuck',
 'edit-warring',
 'bastered==bastered',
 'youbollocks',
 'fart.china',
 '//en.wikipedia.org/wiki/user_talk',
 'concernthanks',
 'ancestryfuck-off-jewish',
 '//www.youtube.com/watch',
 'cellspacing=',
 'j.delanoy',
 'deleted.this',
 'wikilove',
 'sitush',
 'fan-1967',
 'centraliststupid',
 'user_talk',
 'nhrhs2010',
 'wp:3rr',
 'created/took',
 'supertr0ll',
 'neiln',